# Heroes of Pymoli

In [1]:
# import pandas
import pandas as pd

### Defines functions to easily convert formats of columns

In [2]:
'''Creates a function that takes a Dataframe 
and either a column name or a list of column names and converts the format to appropriate to money
'''
def formatMoney(df,colName):
    for col in colName:
        df[col] = df[col].map("${:,.2f}".format)
    return df[colName]

'''Creates a function that takes a Dataframe 
and either a column name or a list of column names and converts the format to appropriate to percentage
'''
def formatPercent(df,colName):
    for col in colName:
        df[col] = df[col].map("{:,.2f}%".format)
    return df[colName]


In [3]:
# Load the file

# File to Load (Remember to Change These)
file_to_load = "Resources/purchase_data.csv"

# Read Purchasing File and store into Pandas data frame
purchase_data = pd.read_csv(file_to_load)

### Player Count

In [4]:
# Count the number of Players (which is the number of rows in dataset)
totalPlayers = purchase_data["SN"].value_counts().count()

# Display it as data frame
totalPlayersDF = pd.DataFrame({"Total Players":[totalPlayers]})

# display results
totalPlayersDF

,Total Players
0,576


### Purchasing Analysis (Total)

In [5]:
# Number of unique items
totalUniqueItems = purchase_data["Item Name"].value_counts().count()

# Average purchase price
averagePurchasePrice = round(purchase_data["Price"].mean(),2)

# Total number of Purchases
totalNumberPurchases = purchase_data["Purchase ID"].count()

# Total Revenue
totalRevenue = purchase_data["Price"].sum()
PurchaseResults = pd.DataFrame({
    "Number of Unique Items":[totalUniqueItems],
    "Average Price": [averagePurchasePrice],
    "Number of Purchases": [totalNumberPurchases],
    "Total Revenue":[totalRevenue]
    
})

# format dataframe
formatMoney(PurchaseResults,["Average Price","Total Revenue"])

# display results
PurchaseResults

,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,179,$3.05,780,"$2,379.77"


### Gender Demographics

In [6]:
# Store unique players and their related info, (SN, Age, Gender) into dataframe
PlayerInfo = purchase_data.iloc[:,1:4]
uniquePlayersInfo = PlayerInfo.drop_duplicates(["SN","Gender"])
uniquePlayersInfoG = uniquePlayersInfo.groupby("Gender")

# Find the counts of each gender
genderCounts = uniquePlayersInfoG.count()
genderCounts = pd.DataFrame(genderCounts["SN"])

# Store each gender count into its own variable
femaleCount = genderCounts.loc["Female"]
maleCount = genderCounts.loc["Male"]
otherCount = genderCounts.loc["Other / Non-Disclosed"]

# convert to percent
malePercent = (maleCount / totalPlayers) * 100
femalePercent = (femaleCount / totalPlayers) * 100
otherPercent = (otherCount / totalPlayers) * 100

genderResults = pd.DataFrame({
    "Total Count": [maleCount[0], femaleCount[0], otherCount[0]],
    "Percentage of Players" : [malePercent[0],femalePercent[0], otherPercent[0]],
    "Gender" : ["Male", "Female","Other / Non-Disclosed"]
})

# format dataframe
formatPercent(genderResults,["Percentage of Players"])

# sets index to gender titles
genderResults = genderResults.set_index("Gender")

# display results
genderResults

,Total Count,Percentage of Players
Gender,,
Male,484,84.03%
Female,81,14.06%
Other / Non-Disclosed,11,1.91%


### Purchasing Analysis (Gender)

In [7]:
# Group by Gender
# Count how many purchases each gender made
purchGend = purchase_data[["Gender","SN"]].groupby("Gender").count()

# Rename SN column to Purchase Count
purchGend = purchGend.rename(columns = {"SN": "Purchase Count"})

# Average purchase price each for gender
purchGend["Average Purchase Price"] =purchase_data[["Gender","Price"]].groupby("Gender").mean()

# Add Total purchase price for each gender
purchGend["Total Purchase Value"] = purchase_data[["Gender","Price"]].groupby("Gender").sum()
purchGend

# Add Avergae total purchase per person
# Group by gender with name and price 
groupGender = purchase_data.groupby(["Gender","SN"])["Price"]
# Average Total Purchae per person for each gender
# Female
femaleMean = groupGender.sum().loc["Female"].mean()
# Male
maleMean = groupGender.sum().loc["Male"].mean()
# Other
othermean = groupGender.sum().loc["Other / Non-Disclosed"].mean()

# add to new column
purchGend["Avg Total Purrchase per Person"] = [femaleMean,maleMean,othermean]

# Format Data Frame
formatMoney(purchGend,["Average Purchase Price","Total Purchase Value","Avg Total Purrchase per Person"])

# Display Summary
purchGend

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purrchase per Person
Gender,,,,
Female,113,$3.20,$361.94,$4.47
Male,652,$3.02,"$1,967.64",$4.07
Other / Non-Disclosed,15,$3.35,$50.19,$4.56


### Age Demographics

In [8]:
# Remove duplicate player names
playerUnique = purchase_data.drop_duplicates(["SN","Age"])

In [9]:
# create bins
bins = [0,9,14,19,24,29,34,39,50]
# create labels
binLabels = ["<10", "10-14", "15-19","20-24","25-29","30-34","35-39","40+"]
#Place data into bins
binData = pd.cut(playerUnique["Age"],bins, labels = binLabels)
binData.value_counts()
# # add this column to dataframe
binData = pd.Series(binData, name ="Age Bracket")

ageBracket = pd.concat([playerUnique,binData],axis = 1)

# Format dataframe
result = pd.DataFrame(ageBracket["Age Bracket"].value_counts()).sort_index()
result["Percentage of Players"] = (result["Age Bracket"] / totalPlayers)*100
formatPercent(result,["Percentage of Players"])

# display output
result

,Age Bracket,Percentage of Players
<10,17,2.95%
10-14,22,3.82%
15-19,107,18.58%
20-24,258,44.79%
25-29,77,13.37%
30-34,52,9.03%
35-39,31,5.38%
40+,12,2.08%


### Purchasing Analysis (Age)

In [10]:
binData2 = pd.cut(purchase_data["Age"],bins, labels = binLabels)
purchase_data["Age Bracket"] = binData2
# Total Count
AgeDemographic = pd.DataFrame(purchase_data.groupby("Age Bracket")["SN"].count())

# # Average Purchase Price
avgPurchase = purchase_data.groupby("Age Bracket")["Price"].mean()

# # Total Purchase value
totalPurchase = purchase_data.groupby("Age Bracket")["Price"].sum()


# Average Total Purchae per person for each Age Bracket
# Group by age bracket
age = purchase_data.groupby(["Age Bracket","SN"])

# Sums the total amount purchased by each player for watch
age2 = age.sum()

# Remove all rows that sum to zero (this means players werent in that age bracket)
age3 = age2[age2["Price"] != 0]

meanList = []

# find mean for each group
for name in binLabels:
    mean = age3["Price"].loc[name].mean()
    meanList.append(mean)

# Merge all results into one data frame
AgeDemographic["Average Purchase Price"] = avgPurchase
AgeDemographic["Total Purchase Value"] = totalPurchase
AgeDemographic["Avg Total Purchase per Person"] = meanList

# Format results
formatMoney(AgeDemographic,["Average Purchase Price","Total Purchase Value","Avg Total Purchase per Person"])

# Print result
AgeDemographic


,SN,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Age Bracket,,,,
<10,23,$3.35,$77.13,$4.54
10-14,28,$2.96,$82.78,$3.76
15-19,136,$3.04,$412.89,$3.86
20-24,365,$3.05,"$1,114.06",$4.32
25-29,101,$2.90,$293.00,$3.81
30-34,73,$2.93,$214.00,$4.12
35-39,41,$3.60,$147.67,$4.76
40+,13,$2.94,$38.24,$3.19


#### Purchasing Analysis (Player)

In [11]:
# Group player
groupPlayer = purchase_data.groupby("SN")["Price"]

# Total spent 
topSpenders = pd.DataFrame(groupPlayer.sum())

# Find top 5 spenders
topSpenders = topSpenders.sort_values(by = "Price",ascending = False).head(5)
names = topSpenders.index
topSpenders
 
# ind average purchase price
t5means = groupPlayer.mean().loc[names]

# topSpenders["Average Purchase Price"] = counts
# Total count of purchases for top 5
t5counts = groupPlayer.count().loc[names]
t5counts
# Create output, rename columns 
topSpenders["Purchase Count"] = t5counts
topSpenders["Average Purchase Price"] = t5means
topSpenders = topSpenders.rename(columns = {"Price": "Total Purchase Value"})

# change the order
topSpenders = topSpenders[["Purchase Count","Average Purchase Price","Total Purchase Value"]]

# change the format
formatMoney(topSpenders,["Average Purchase Price","Total Purchase Value"])

# Print the output
topSpenders

,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,
Lisosia93,5,$3.79,$18.96
Idastidru52,4,$3.86,$15.45
Chamjask73,3,$4.61,$13.83
Iral74,4,$3.41,$13.62
Iskadarya95,3,$4.37,$13.10


#### Item Analysis

In [12]:
# Item info
itemInfo = purchase_data[["Item ID", "Item Name", "Price"]]

# Group by item id and item name
groupedItems = itemInfo.groupby(["Item ID", "Item Name"])

# Total purchase count
count = groupedItems.count()

# Price  = sum of purchase price for each item / purchase count
price = (groupedItems.sum()/groupedItems.count())

# put all in one data frame
count["Item Price"] = price["Price"]

# total value = price * purchase count
count["Total Purchase Value"] = count["Price"] * count["Item Price"]

# Create a copy of data frame for the next question
mostProfitable = count
# format nicely
# rename columns
count = count.rename(columns = {"Price": "Purchase Count"})

# change format
formatMoney(count, ["Item Price","Total Purchase Value"])


#Display dataframe 
popularItems= count.sort_values(by = "Purchase Count", ascending = False)
# Show top 5 items
popularItems.head(5)

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
92,Final Critic,13,$4.61,$59.99
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
145,Fiery Glass Crusader,9,$4.58,$41.22
132,Persuasion,9,$3.22,$28.99
108,"Extraction, Quickblade Of Trembling Hands",9,$3.53,$31.77


### Most Profitable Items

In [13]:
# sort by total purchase value
sortedMostProfit = mostProfitable.sort_values(by = "Total Purchase Value", ascending = False)

# change to format
formatMoney(sortedMostProfit, ["Item Price","Total Purchase Value"])
# Print top 5
sortedMostProfit.head(5)

,,Price,Item Price,Total Purchase Value
Item ID,Item Name,,,
92,Final Critic,13,$4.61,$59.99
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
82,Nirvana,9,$4.90,$44.10
145,Fiery Glass Crusader,9,$4.58,$41.22
103,Singed Scalpel,8,$4.35,$34.80


### Report
#### Gender analysis
From the analysis of the geneders of players who play Heroes Of Pymoli, we can see that majority of the players were male (84.03%), followed by females (14.06%) and then other/Non-disclosed (1.91%). Males as a group spent the most money (\\$1,967.64) on the game, but this could also be because there were approximatley 6 times more male purchases than female purchase and and about 43 times more than other/non-disclosed purchases.

Although males spent the most in total, they were the lowest group in regards to average purchase price (\\$3.20) and average total purchase per person (\\$4.07). This means that on average males bought cheaper items than compared to the other two groups, with female and other/ non-dislcosed buying more expensive items than what males appeared to buy (the average purchase prices being (\\$3.20 and \\$3.35 respectivley). Comparing average total purchase prices per person shows us that a male player spends less in total on items in the game than compared to the other two groups, with once again a player of gender other/non-dislcosed spending the most.

#### Age Analysis
Majority of players were aged between 20 and 24 with 44.79% of players being within this age bracket. Players age 40+ were in the lowest percentage (2.08%) followed closely by the age bracket <10 (2.85%).

Players in the age bracket of 20-24 spent the most in total (\\$1,114.06), with players aged 40+ spending the least in total. Players aged 35 to 39 had the highest average purchase price (\\$3.60), suggesting that they bought items with higher prices. This was followed by players aged <10 (\\$3.35). The highest average purchase price per person was for the age bracket 35-39 (\\$4.76), this suggests that players aged 35-39 spent more on the game than the other age barckets.

### Profitable Items Analysis
The item 'Final Critic' was bought the most (13 times) and had the highest price out of the top 5 most purchased items. This item also had the highest total purchase value, making it the most popular and profitable item.